Updates to script for version 2

1.   Using PySpark & Spark Pandas API for better performance & speed
2.   Using Azure Cousmos DB to send cleaned data to PowerBI Dashboard
3.   Minor usage of Numba Framework to speed up some methods within this script
4.   Potential Trials of Time-Series Machine Learning Modeling of Data 
5.   Trials with using the Libra Library for potential simple Machine Learning development
6.   Determine if using Azure Synapse makes sense for this project for giving no-code manipulation of the data

Website Links & Documentation for all program enhancements

1.   PySpark Examples - https://github.com/apache/spark/tree/f74867bddf/examples/src/main/python
2.   Azure Cosmos DB - https://docs.microsoft.com/en-us/azure/cosmos-db/sql/create-sql-api-python
2.   Azure Cosmos DB Github Demo - https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/main/cosmos_get_started.py
3.   Numba - https://numba.pydata.org/
4.   Time-Series Machine Learning Algos - https://www.advancinganalytics.co.uk/blog/2021/06/22/10-incredibly-useful-time-series-forecasting-algorithms
5.   Libra - https://libradocs.org/
6.   Azure Synapse & Cosmos DB Integration - https://docs.microsoft.com/en-us/azure/cosmos-db/synapse-link
7.   Spark SQL & Python - https://spark.apache.org/docs/latest/sql-programming-guide.html
8.   DeepHaven Streaming Dataframe - https://deephaven.io/?utm_term=streaming%20data&utm_campaign=Website+Traffic+Q4+2022&utm_source=adwords&utm_medium=ppc&hsa_acc=4673439537&hsa_cam=18322365595&hsa_grp=139855244374&hsa_ad=621564846172&hsa_src=g&hsa_tgt=kwd-161093182&hsa_kw=streaming%20data&hsa_mt=p&hsa_net=adwords&hsa_ver=3
9.   Spark SQL & Python - https://spark.apache.org/docs/latest/
10.  Google Sheets & Python - https://ploomber.io/blog/gsheets/

In [11]:
#all imports 
import gspread
import numpy as np
import pyspark
import pyspark.pandas as ps
import numba
from numba import jit
import os
import json
from datetime import *

In [12]:
#setup spark env & import spark variables
from pyspark.sql.types import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [33]:
#call to the dataset within Google Sheets
sa = gspread.service_account(filename="/com.docker.devenvironments.code/kuthesisdataclean-558333f9362c.json")
sheet = sa.open("POU_Unclean_Data")
work_sheet = sheet.worksheet("POU")
rows = work_sheet.get_all_values()
#rows = rows.get_all_records()
df = spark.createDataFrame(rows)

In [ ]:
df.printSchema()

22/09/28 22:07:07 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1422574 ms exceeds timeout 120000 ms
22/09/28 22:07:07 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
#Format the dataframe so we can start working on using it for data analysis
#Next we are going to format the rows to allow for the data to get into a data frame
"""
df  = df.drop(labels=0, axis=0)
df  = df.reset_index(drop=True)
df.columns = df.loc[0]
df  = df.loc[1:].reset_index(drop=True)
df.drop_duplicates(subset="Serial Number", keep='last')
df
"""

In [34]:
@jit
def format_df(df):
    df = df.drop(labels=0, axis=0)
    df = df.index(drop=True)
    df.columns = df.collect()[0]
    df = df[1:].collect(drop=True)
    df.drop_duplicates(subset="Serial Number", keeps='last')

In [35]:
df = format_df(df)

/tmp/ipykernel_1155/472170987.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "format_df" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_1155/472170987.py (3)

File "tmp/ipykernel_1155/472170987.py", line 3:
<source missing, REPL/exec in use?>

  @jit
/home/vscode/micromamba/envs/py39/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "format_df" was compiled in object mode without forceobj=True.

File "tmp/ipykernel_1155/472170987.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/vscode/micromamba/envs/py39/lib/python3.10/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/en/

TypeError: DataFrame.drop() got an unexpected keyword argument 'labels'